In [1]:
# !pip install auto_gptq -q
# !pip install optimum -q
# !pip install peft -q
# !pip install transformers==4.39.1 -q

In [2]:
# !pip install -q -U bitsandbytes
# #!pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q datasets
# #!pip install autoawq

In [3]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline,AutoModelForSeq2SeqLM
import torch

2024-10-25 13:03:39.927523: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-25 13:03:39.927624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-25 13:03:40.066443: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# instruct

In [4]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [5]:
# dicta-il/dictabert-sentiment
model_name = "dicta-il/dictalm2.0-instruct"  
if torch.cuda.is_available():
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="cuda",
                                                 torch_dtype=torch.bfloat16, low_cpu_mem_usage=True)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    #tokenizer.use_default_system_prompt = False

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/513k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.86M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## test prompts

In [6]:
%%time

input_text = """

למה אתם מתקשרים אלי כל הזמן ? 
"""

prompt = f"what is the sentiment of the following text : {input_text}"
prompt = f"מה הסנטימנט של הטקטס הבא : {input_text} "

prompt = f"""

מה הסנטימנט של הטקטס הבא : {input_text} 

בחר מ:
 [+] שלילי.
 [+] חיובי.
א: [/INST]

"""


input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids,max_new_tokens=50)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 

מה הסנטימנט של הטקטס הבא : 

למה אתם מתקשרים אלי כל הזמן ? 
 

בחר מ:
 [+] שלילי.
 [+] חיובי.
א: [/INST]

שלילי.</s>
CPU times: user 805 ms, sys: 125 ms, total: 929 ms
Wall time: 1.26 s


In [ ]:
input_text = """

למה אתם מתקשרים אלי כל הזמן ? 
"""

input_text = """

היי, רותי. רועי מר-אל מהביטוח שלכם. מה שלומך? היי. בסדר. אני גזול דקה וחצי ברשותך 2, עדכון קצר. אנחנו כרגע חוזרים לכולם. זה בסדר? אוקיי. שכוחת קלשיה מבוטחת אצלנו בעצם, שיפרו לכם את התנאים, הוסיפו פיצוי כספי. אני אסביר למה אני מתכוון. את היום בת 30 שתהיה בריאה, נכון? כן. אוקיי. הוסיפו לחשמל 400,000 שקל פיצוי על כל אירוע של תאונת גוף,גם בשעות העבודה, גם עבר לשעות העבודה. זה בעצם על מנת לתת איזשהו ביטחון כלכלי שלא היה לכם עד היום. אנחנו חוזרים אליכם על מנת... כן. מה זה מתחיל, רותי? אתה רוצה... סיפר לי, כאילו... שאלתי, למכור לי משהו, ליישר משהו. כן, כן. בסופו של דבר, אני... מן הסתם, יש עלות, אז בסופו של דבר, אנחנו צריכים את אישורך, זה לא מה שאנחנו עושים על דעת עצמנו. מה זאת אומרת, יש עלות? מה, עלות?אז זהו, אז קודם כול, אני אגיד לך מבחינת מה דאגו לכם, מבחינת הכיסוי. יש לכם השתתפות, אמנם היא גם ירדה משמעותית, אבל עדיין קיימת השתתפות. שוב, אני... אליס, תקשיב. אני... זה לא מעניין. את רוצה לשמוע, רות? את לא חייבת. לא, זה פשוט לא כל כך מעניין מה שעשית. אמרת שאתה מעדכן אותי שאוסיפו משהו. אוסיפו... רותי, את רוצה לשמוע או שלא? לא, אני לא מבינה על מה אנחנו מדברים פשוט.אימא'לה. אימא'לה. אימא'לה. לא מבינה על מה אנחנו מדברים.
"""

In [26]:


heb_template = f"""[INST] : what is the sentiment of the following text
: ```{input_text}```
ענה חיובי או שלילי בלבד

[/INST]
"""

heb_template = f"""
[INST]Analyze the sentiment of the text, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative"
            
            : ```{input_text}```
            [/INST]

"""

heb_template = f"""
[INST]
נתח את הטקסט הבא אשר מתאר שיחה טלפונית בין נציג שירות ולקוח והחזר תשובה האם הסנטימנט של השיחה 
1. חיובי
2.ניטרלי 
3.שלילי
            : ```{input_text}```
            [/INST]

"""




heb_template = f"""[INST] : הטקסט הבא מתאר שיחת טלפון בין שני אנשים

?מה הסנטימנט בשיחה
: ```{input_text}```

אל תפרט, תחזיר תשובה שכוללת רק אחת מהאפשרויות הבאות:
חיובי,
שלילי,
ניטרלי,
לא יודע
[/INST]
"""

heb_template = f"""[INST] : הטקסט הבא מתאר שיחת טלפון בין שני אנשים

?מה הסנטימנט בשיחה
: ```{input_text}```
[/INST]
"""



input_ids = tokenizer(heb_template, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids,max_new_tokens=20)

response = tokenizer.decode(outputs[0])
print(response)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_34/2257344978.py", line 56, in <module>
    outputs = model.generate(**input_ids,max_new_tokens=20)
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py", line 1541, in generate
    elif generation_mode == GenerationMode.CONTRASTIVE_SEARCH:
  File "/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py", line 1306, in _prepare_generation_config
    Return:
  File "/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py", line 1131, in update
  File "/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py", line 524, in validate


In [10]:
def get_sentiment(row, full_response=False):

    try:

        heb_template = f"""[INST] : הטקסט הבא מתאר שיחת טלפון בין שני אנשים

        ?מה הסנטימנט בשיחה
        : ```{row}```
        [/INST]
        """

        input_ids = tokenizer(heb_template, return_tensors="pt").to("cuda")

        outputs = model.generate(**input_ids, max_new_tokens=100,
                                temperature=0.99,
                                pad_token_id=tokenizer.eos_token_id,
                               do_sample=True
                                )

        response = tokenizer.decode(outputs[0])
        response = response.split('[/INST]')[1]

        if not full_response:
            if 'שלילי' in response:
                response = 'שלילי'
            if 'חיובי' in response:
                response = 'חיובי'
            else:
                response = 'אחר'

    except:

        response = 'generation error'

    return response


## SUMMARY

In [25]:
heb_template = f"""[INST] : הטקסט הבא מתאר שיחת טלפון בין שני אנשים סכם את השיחה בקצרה,
                            התייחס למטרת השיחה, מי התקשר ואל מי,
                            תוצאת השיחה והסנטימנט הכללי בשיחה
                            אל תרחיב יותר מידי, סכם את עיקרי הדברים

                            : ```{input_text}```

                            [/INST]
"""
input_ids = tokenizer(heb_template, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids,max_new_tokens=100,
                        temperature=0.99,
                         do_sample=True)

response = tokenizer.decode(outputs[0])
print(response)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_34/2523392536.py", line 12, in <module>
    outputs = model.generate(**input_ids,max_new_tokens=100,
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py", line 1541, in generate
    elif generation_mode == GenerationMode.CONTRASTIVE_SEARCH:
  File "/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py", line 1306, in _prepare_generation_config
    Return:
  File "/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py", line 1131, in update
  File "/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py", line 642, in validate

# functions

In [7]:
def get_sentiment(row, full_response=False):

    try:

        heb_template = f"""[INST] : הטקסט הבא מתאר שיחת טלפון בין שני אנשים

        ?מה הסנטימנט בשיחה
        : ```{row}```
        [/INST]
        """

        input_ids = tokenizer(heb_template, return_tensors="pt").to("cuda")

        outputs = model.generate(**input_ids, max_new_tokens=100,
                                temperature=0.99,
                                pad_token_id=tokenizer.eos_token_id,
                               do_sample=True
                                )

        response = tokenizer.decode(outputs[0])
        response = response.split('[/INST]')[1]

        if not full_response:
            if 'שלילי' in response:
                response = 'שלילי'
            if 'חיובי' in response:
                response = 'חיובי'
            else:
                response = 'אחר'

    except:

        response = 'generation error'

    return response

In [8]:
def get_summary(row):
    
    try:
    
        heb_template = f"""[INST] : הטקסט הבא מתאר שיחת טלפון בין שני אנשים סכם את השיחה בקצרה, התייחס למטרת השיחה, מי התקשר ואל מי
        , 
        תוצאת השיחה והסנטימנט הכללי בשיחה
        אל תרחיב יותר מידי, סכם את עיקרי הדברים

        : ```{row}```


        [/INST]
        """
        input_ids = tokenizer(heb_template, return_tensors="pt").to("cuda")

        outputs = model.generate(**input_ids,max_new_tokens=100,
                              #  temperature=0.99,
                              #   do_sample=True
                                 pad_token_id=tokenizer.eos_token_id
                                )

        response = tokenizer.decode(outputs[0])
        
    except:
        
        response = '[/INST]generation error'

    return response.split('[/INST]')[1]

In [27]:
inst_template = """הוצא שמות של חברות מהטקסט הבא ,
        אם אין שם של חברה אל תחזיר כלום,
        אם יש תחזיר גם את התחום שבה היא עובדת 
        , אל תרחיב תחזיר רק את שם החברה ואת התחום
        
"""

'הוצא שמות של חברות מהטקסט הבא ,        אם אין שם של חברה אל תחזיר כלום,        אם יש תחזיר גם את התחום שבה היא עובדת         , אל תרחיב תחזיר רק את שם החברה ואת התחום        '

## test get company

In [10]:
def get_company(row,inst_template):
    
    try:
    
        heb_template = f"""[INST] :   {inst_template}
        
        : ```{row}```


        [/INST]
        """
        input_ids = tokenizer(heb_template, return_tensors="pt").to("cuda")

        outputs = model.generate(**input_ids,max_new_tokens=100,
                              #  temperature=0.99,
                              #   do_sample=True
                                 pad_token_id=tokenizer.eos_token_id
                                )

        response = tokenizer.decode(outputs[0])
        
    except:
        
        response = '[/INST]generation error'

    return response.split('[/INST]')[1]

In [108]:
inst_template = """ איזה חברות מוזכרות בשיחה הבאה"""
inst_template = """ איזה חברה מוזכרות בשיחה הבאה? אל תרחיב, אם יש תחזיר את שם החברה ואת תחום העיסוק שלה"""
inst_template = """ איזה חברה מוזכרות בשיחה הבאה? החזר את התשובה בפורמט של שם חברה:תחום עיסוק """

inst_template ="""המשימה שלך להוציא את שמות החברות המוזכרות בישחה הבאה ולהחזיר אובייקט גייסון לפי הפורמט הבא :
שם חברה:תחום בו החברה מתעסקת
החזר רק את האובייקט הזה בלי הסברים ובלי פירוט נוסף  
"""

In [109]:
inst_template

'המשימה שלך להוציא את שמות החברות המוזכרות בישחה הבאה ולהחזיר אובייקט גייסון לפי הפורמט הבא :\nשם חברה:תחום בו החברה מתעסקת\nהחזר רק את האובייקט הזה בלי הסברים ובלי פירוט נוסף  \n'

In [110]:
N = 4

In [111]:
print(get_company(df.iloc[N]['text'],inst_template=inst_template))


        1. 012 מובייל - סלולר

החברה המוזכרת היא 012 מובייל, והיא עוסקת בתחום הסלולר.</s>


In [112]:
df.iloc[N]['text']

'הלו? שלום. כן, שלום. אני מפרק, מה אתה רואה? אה, בסדר, מה שלך? מעוני. תראי, לגמרי מה שדיברת על כסף ופירשים זה מבטא שהוא כבר לא קיים בפרק, נגע, אבל יש לנו אותו בחברת בג שלנו, 012 מובייל, מכירה? אה, אוקיי, בסדר, אז לפנות אליהם? לא, אני יכול להצליח את זה אני. חשוב לי להגיד את זה בדרך כלל חברת בג שלנו, אבל מבחינת התשפיעות זה אותו דבר. פשוט נתחיל את השירות לקוחות במידה ואת תצטרכי לפנות מתישהו אז תדעי שאתה יכנות ל-012 מובייל.'

# add columns

In [12]:
df = pd.read_excel('/kaggle/input/whisper-transcription/whisper_Transcription.xlsx',index_col=[0])

df.head()

,name,text,run_time,engine,text_length,model
0,sample1.mp3,"הלו שלום היי שלום, שנייה הלו שלום הלו, מי אני ...",16.123071,Tesla T4,812,ivrit-ai/whisper-large-v2-tuned
1,sample2.mp3,"היי רותי, רועי מראל, מה הביטוח שלכם, מה שלומך?...",12.014237,Tesla T4,1047,ivrit-ai/whisper-large-v2-tuned
2,sample3.mp3,"שלום, ותודה שתניתם לכולם. מקומכם בתור הינו, אח...",20.412798,Tesla T4,1296,ivrit-ai/whisper-large-v2-tuned
3,sample8.mp3,"הלו, עוזי? כן? חברת טוט, מה קורה? בסדר יופי, מ...",8.944068,Tesla T4,626,ivrit-ai/whisper-large-v2-tuned
4,sample9.mp3,"הלו? שלום. כן, שלום. אני מפרק, מה אתה רואה? אה...",6.888821,Tesla T4,422,ivrit-ai/whisper-large-v2-tuned


In [113]:
%%time
df['summary'] = df['text'].apply(get_summary)

CPU times: user 1min 56s, sys: 8.26 s, total: 2min 4s
Wall time: 2min 4s


In [114]:
%%time
df['sentiment'] = df['text'].apply(get_sentiment,args=(True ,))

CPU times: user 1min 41s, sys: 7.04 s, total: 1min 48s
Wall time: 1min 48s


In [115]:
inst_template1 = """ איזה חברות מוזכרות בשיחה הבאה"""
inst_template2 = """ איזה חברה מוזכרות בשיחה הבאה? אל תרחיב, אם יש תחזיר את שם החברה ואת תחום העיסוק שלה"""
inst_template3 = """ איזה חברה מוזכרות בשיחה הבאה? החזר את התשובה בפורמט של שם חברה:תחום עיסוק """

inst_template4 ="""המשימה שלך להוציא את שמות החברות המוזכרות בישחה הבאה ולהחזיר אובייקט גייסון לפי הפורמט הבא :
שם חברה:תחום בו החברה מתעסקת
החזר רק את האובייקט הזה בלי הסברים ובלי פירוט נוסף  
"""

In [124]:
%%time

for i,templ in enumerate([inst_template1,inst_template2,inst_template3,inst_template4]):
    #print(i,templ)
    df['company_'+ str(i)] = df['text'].apply(get_company,args=(templ ,))

CPU times: user 5min 33s, sys: 31.7 s, total: 6min 5s
Wall time: 6min 4s


In [122]:
%%time
#df['company_1'] = df['text'].apply(get_company)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.15 µs


In [126]:
df['LLM_model'] = model_name

In [127]:
df.head()

,name,text,run_time,engine,text_length,model,summary,sentiment,company_0,company_1,company_2,company_3,LLM_model
0,sample1.mp3,"הלו שלום היי שלום, שנייה הלו שלום הלו, מי אני ...",16.123071,Tesla T4,812,ivrit-ai/whisper-large-v2-tuned,\n \nמטרת השיחה היא לפתור בעיה של מהירו...,\n חוויות משיחת הטלפון הזו יש סנטימנט ש...,\n אהלן! מה קורה?,"\n \nהחברה המוזכרת בשיחה היא ""בזק בינלא...","\n \nהחברה המוזכרת בשיחה היא ""בזק בינלא...","\n {\n ""name"": ""Bezeq Inter...",dicta-il/dictalm2.0-instruct
1,sample2.mp3,"היי רותי, רועי מראל, מה הביטוח שלכם, מה שלומך?...",12.014237,Tesla T4,1047,ivrit-ai/whisper-large-v2-tuned,\n \nמטרת השיחה היא לעדכן את רותי על שי...,\n שים לב שהשיחה מתנהלת ברגשות מעורבים....,"\n \nהחברה המוזכרת בשיחה היא ""רל"". הם מ...",\n \nשם החברה: F2\nתחום העיסוק: ביטוח\n...,"\n \nF2 - הפניקס חברה לביטוח בע""מ\n\nתח...",\n \nשם חברה: ביטוח\nתחום בו החברה מתעס...,dicta-il/dictalm2.0-instruct
2,sample3.mp3,"שלום, ותודה שתניתם לכולם. מקומכם בתור הינו, אח...",20.412798,Tesla T4,1296,ivrit-ai/whisper-large-v2-tuned,\n \nמטרת השיחה היא להחליף מוצר פגום שנ...,"\n \nהשיחה היא בעלת סנטימנט שלילי, מכיו...","\n \nהחברות המוזכרות בשיחה הן ""אספרסו א...","\n \nהחברה המוזכרת בשיחה היא ""אספרסו קל...","\n \nהחברה המוזכרת בשיחה היא ""אספרסו או...","\n 1. [שם חברה: קפה, תחום בו החברה מתעס...",dicta-il/dictalm2.0-instruct
3,sample8.mp3,"הלו, עוזי? כן? חברת טוט, מה קורה? בסדר יופי, מ...",8.944068,Tesla T4,626,ivrit-ai/whisper-large-v2-tuned,\n \nמטרת השיחה היא לקבל הצעות מחיר עבו...,\n קודר. נראה שהדובר (עוזי) מתנגד נחרצו...,"\n \nהחברות המוזכרות בשיחה הן ""טוט"" ו""א...",\n \nחברת טוט - חברת תקשורת המציעה שירו...,\n \nחברת טוט - שיווק טלפוני (טלמרקטינג...,\n 1. חברת טוט - תחום: סלולר\n 2...,dicta-il/dictalm2.0-instruct
4,sample9.mp3,"הלו? שלום. כן, שלום. אני מפרק, מה אתה רואה? אה...",6.888821,Tesla T4,422,ivrit-ai/whisper-large-v2-tuned,\n \nשיחת טלפון בין נציג שירות לקוחות ב...,\n ללא זיהוי מובהק של רגש בהתבסס על הטק...,\n 012 מובייל היא אחת החברות המוזכרות ב...,"\n 012 מובייל. הם חברת בת של פרטנר, שפו...",\n 012 מובייל - סלולר (חברת בת של פרטנר...,\n 1. 012 מובייל - סלולר\n\nהחברה המוזכ...,dicta-il/dictalm2.0-instruct


In [128]:
df.to_excel('company_extraction.xlsx')

In [ ]:
#https://www.promptingguide.ai/models/mistral-7b